## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [28]:
%matplotlib inline
%reload_ext nb_black

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
nba_df = pd.read_csv('data/nba_player_seasons.csv')




<IPython.core.display.Javascript object>

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [3]:
# answer goes here
nba_df = nba_df.loc[nba_df['GS'] >= 20]
nba_df = nba_df.loc[nba_df['MP'] >= 10]




<IPython.core.display.Javascript object>

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [4]:
# answer goes here
cols_of_interest = [
    'PTS',
    'TRB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'MP',
    'FT%',
    '3P',
]

nba_interest = nba_df[cols_of_interest]




<IPython.core.display.Javascript object>

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [5]:
# answer goes here
scaler = StandardScaler()
scaled = scaler.fit_transform(nba_interest)
scaled




array([[ 0.11463777,  1.6129094 , -0.61781554, ...,  0.98199392,
        -2.58644655, -1.3535285 ],
       [-0.7449851 ,  0.76862715, -0.33088552, ..., -0.90364619,
        -0.25522074, -1.3535285 ],
       [ 1.38687962,  1.49778   , -0.23524218, ...,  0.94465452,
         0.85583157, -1.24053537],
       ...,
       [-0.2635963 ,  0.42323896, -0.71345889, ...,  0.14185724,
         0.71695003,  0.79334113],
       [ 0.32094726,  1.42102707, -0.5221722 , ...,  0.90731511,
        -1.15795073, -1.24053537],
       [-0.77937002,  0.34648603, -0.90474557, ..., -0.71694914,
        -0.06673865, -1.3535285 ]])

<IPython.core.display.Javascript object>

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [20]:
# answer goes here
gauss3 = GaussianMixture(n_components=3, random_state=42)
gauss3.fit(scaled)
gauss3_avg = pd.DataFrame(scaler.inverse_transform(gauss3.means_), columns=nba_interest.columns)



kmeans3 = KMeans(n_clusters=3, random_state=42)
kmeans3.fit(scaled)
kmeans3_avg = pd.DataFrame(
        scaler.inverse_transform(kmeans3.cluster_centers_), columns=nba_interest.columns
    )


<IPython.core.display.Javascript object>

In [24]:
gauss3_avg.style.background_gradient(axis=0)

,PTS,TRB,AST,STL,BLK,TOV,MP,FT%,3P
0,10.144020,4.197849,1.701457,0.792696,0.394321,1.141280,25.481773,0.764839,1.221734
1,16.787175,4.626279,4.943632,1.180902,0.395330,2.311917,31.398888,0.802250,1.653905
2,13.785906,8.301836,2.115263,0.780595,1.242260,1.761330,28.325139,0.692891,0.492237


<IPython.core.display.Javascript object>

In [25]:
gauss7 = GaussianMixture(n_components=7, random_state=42)
gauss7.fit(scaled)
gauss7_avg = pd.DataFrame(scaler.inverse_transform(gauss7.means_), columns=nba_interest.columns)



kmeans7 = KMeans(n_clusters=7, random_state=42)
kmeans7.fit(scaled)
kmeans7_avg = pd.DataFrame(
        scaler.inverse_transform(kmeans7.cluster_centers_), columns=nba_interest.columns
    )


<IPython.core.display.Javascript object>

In [27]:
gauss7_avg.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,MP,FT%,3P
0,7.353222,4.082407,1.223798,0.720552,0.453837,0.882791,21.820562,0.712158,0.748324
1,13.622095,5.528638,1.922538,0.895615,0.476239,1.397004,29.913085,0.800515,1.588170
2,12.034762,4.551974,4.896158,1.245015,0.448274,2.184469,29.355140,0.743463,1.333081
3,21.994594,5.214224,5.855807,1.374110,0.436163,2.830199,34.252993,0.815770,1.996330
4,9.988144,8.043646,1.366459,0.658250,1.085476,1.382581,24.817138,0.638437,0.000000
5,17.841918,8.384793,2.949724,0.881223,1.435481,2.106013,30.891099,0.780532,1.032807
6,11.800594,2.835238,3.118724,0.826029,0.206435,1.509132,27.225362,0.806759,1.472513


<IPython.core.display.Javascript object>

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [42]:
# answer goes here
gauss3_prob_df = pd.DataFrame(gauss3.predict_proba(scaled))

# prob = np.array([1,1,1])
# entropy([1,1,1])
# sum(prob * np.log2(prob))


entropy(gauss3_prob_df.T)
gauss3_prob_df['entropy'] = gauss3_prob_df.apply(entropy, axis=1)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
top_5_df = gauss3_prob_df.sort_values(by="entropy").head(5)
bottom_5_df = gauss3_prob_df.sort_values(by="entropy").tail(5)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
nba_interest.iloc[top_5_df.index]

,PTS,TRB,AST,STL,BLK,TOV,MP,FT%,3P
2117,14.2,11.8,0.4,0.6,3.7,1.9,29.1,0.650,0.0
1218,20.2,7.8,2.1,0.9,2.5,3.8,25.4,0.783,1.2
484,13.3,7.2,1.6,0.8,2.7,1.4,28.6,0.736,1.0
651,28.1,11.1,2.3,1.5,2.6,2.2,36.4,0.828,0.7
940,22.7,6.6,1.2,0.8,2.4,1.9,32.4,0.793,1.9


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
kmeans3_avg

,PTS,TRB,AST,STL,BLK,TOV,MP,FT%,3P
0,9.254802,3.893220,1.859887,0.729096,0.391525,1.127119,24.343503,0.760740,1.088701
1,13.220958,8.589820,1.867066,0.793413,1.247904,1.658683,28.398204,0.684940,0.374251
2,18.216961,5.110247,4.787633,1.236042,0.455477,2.385866,32.737102,0.805435,1.820495


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>